In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentences = [
    "The quick brown fox",
    "Jumps over the lazy dog",
    "Pytorch is an amazing",
    "Framework for deep learning",
    "Natural Language Processing is",
    "Great way to work with text"
]

In [3]:
words = set(word for sentence in sentences for word in sentence.split())
vocab = sorted(list(words))
word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for word, i in word_to_ix.items()}

In [4]:
def make_data(sentences):
    data = []
    for sentence in sentences:
        tokens = sentence.split()
        for i in range(len(tokens) - 1):
            input_seq = tokens[:i+1]
            target = tokens[i+1]
            data.append((
                torch.tensor([word_to_ix[w] for w in input_seq], dtype=torch.long),
                torch.tensor(word_to_ix[target], dtype=torch.long)
            ))
    return data

training_data = make_data(sentences)


In [5]:
class NextWordPredictor(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(NextWordPredictor, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x):
        embeds = self.embedding(x)
        output, hidden = self.rnn(embeds)
        out = self.fc(output[:, -1, :])  # take last hidden state
        return out

In [6]:
vocab_size = len(vocab)
embed_dim = 10
hidden_dim = 20
model = NextWordPredictor(vocab_size, embed_dim, hidden_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [7]:
for epoch in range(300):
    total_loss = 0
    for seq, target in training_data:
        optimizer.zero_grad()
        output = model(seq.unsqueeze(0))
        loss = criterion(output, target.unsqueeze(0))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if (epoch+1) % 100 == 0:
        print(f"Epoch [{epoch+1}/300], Loss: {total_loss:.4f}")

Epoch [100/300], Loss: 0.0322
Epoch [200/300], Loss: 0.0070
Epoch [300/300], Loss: 0.0021


In [8]:
def predict_next_word(model, input_text):
    model.eval()
    with torch.no_grad():
        tokens = input_text.split()
        input_seq = torch.tensor([word_to_ix[w] for w in tokens if w in word_to_ix], dtype=torch.long).unsqueeze(0)
        output = model(input_seq)
        predicted_ix = torch.argmax(output, dim=1).item()
        return ix_to_word[predicted_ix]

In [9]:
test_input = "Pytorch is an"
predicted_word = predict_next_word(model, test_input)
print(f"Input: '{test_input}' → Predicted Next Word: '{predicted_word}'")

Input: 'Pytorch is an' → Predicted Next Word: 'amazing'


In [10]:
sentences

['The quick brown fox',
 'Jumps over the lazy dog',
 'Pytorch is an amazing',
 'Framework for deep learning',
 'Natural Language Processing is',
 'Great way to work with text']